In [7]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import torch
import time
import sys
import resource
from torch.utils.data import DataLoader
from bnn import BayesianNeuralNetwork
from pyro.infer import SVI, Trace_ELBO
from pyro.infer.autoguide import AutoDiagonalNormal
from pyro.infer import Predictive
import pyro

from data_classes import *
from read_input import *
from read_trainset import *
from network import *
from prepare_batches import *
from traininit import *
from data_set import *
from data_loader import *
from optimization_step import *
from output_nn import *
from py_aeio import *

/nethome/farri002/miniconda3/envs/tyxe/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = "cpu"

In [10]:
tin_file = "/nethome/farri002/mlp-pt-au-o-h/ml-construction/PdO/desc-20-8/NN-20-20/train.in"
tin = read_train_in(tin_file)
torch.manual_seed(tin.pytorch_seed)
np.random.seed(tin.numpy_seed)
if tin.verbose: io_input_reading(tin)
tin.train_file = '/nethome/farri002/mlp-pt-au-o-h/ml-construction/PdO/desc-20-8/data.train.ascii'
tin.train_forces_file = '/nethome/farri002/mlp-pt-au-o-h/ml-construction/PdO/desc-20-8/data.train.forces'

In [11]:
list_structures_energy, list_structures_forces, list_removed, max_nnb, tin = read_list_structures(tin)
N_removed = len(list_removed)
N_struc_E = len(list_structures_energy)
N_struc_F = len(list_structures_forces)

In [12]:
def select_batches(tin, trainset_params, device, list_structures_energy, list_structures_forces,
				   max_nnb, N_batch_train, N_batch_valid, train_sampler_E, valid_sampler_E):
        """
        Select which structures belong to each batch for training.
        Returns: four objects of the class data_set_loader.PrepDataloader(), for train/test and energy/forces
        """
        dataset_energy = StructureDataset(list_structures_energy, tin.sys_species, tin.networks_param["input_size"], max_nnb)
        
        dataset_energy_size = len(dataset_energy)

        # Normalize
        E_scaling, E_shift = tin.trainset_params.E_scaling, tin.trainset_params.E_shift
        sfval_avg, sfval_cov = tin.setup_params.sfval_avg, tin.setup_params.sfval_cov
        dataset_energy.normalize_E(trainset_params.E_scaling, trainset_params.E_shift)
        stp_shift, stp_scale = dataset_energy.normalize_stp(sfval_avg, sfval_cov)

        # Split in train/test
        #train_sampler_E, valid_sampler_E = split_database(dataset_energy_size, tin.test_split)

        train_energy_data = PrepDataloader(dataset=dataset_energy, train_forces=False, N_batch=N_batch_train,
                                        sampler=train_sampler_E, memory_mode=tin.memory_mode, device=device, dataname="train_energy")
        valid_energy_data = PrepDataloader(dataset=dataset_energy, train_forces=False, N_batch=N_batch_valid,
                                        sampler=valid_sampler_E, memory_mode=tin.memory_mode, device=device, dataname="valid_energy")

        return train_energy_data, valid_energy_data

In [13]:
def get_train_test(grouped_train_loader):
    for data_batch in grouped_train_loader:
        grp_descrp, grp_energy, logic_reduce = data_batch[0][10], data_batch[0][11], data_batch[0][12]

    grp_descrp[0] = grp_descrp[0].float()
    grp_descrp[1] = grp_descrp[1].float()

    logic_reduce[0] = logic_reduce[0].float()
    logic_reduce[1] = logic_reduce[1].float()

    return grp_descrp, grp_energy, logic_reduce

In [14]:
def get_sets_from_indices(training_indices, test_indices, list_structures_energy):
    N_batch_train, N_batch_valid = 1, 1 #select_batch_size(tin, list_structures_energy, list_structures_forces)

    # Join datasets with forces and only energies in a single torch dataset AND prepare batches
    train_energy_data, valid_energy_data = select_batches(tin, tin.trainset_params, device, list_structures_energy, None,
                    max_nnb, N_batch_train, N_batch_valid, training_indices, test_indices)
    
    grouped_train_data = GroupedDataset(train_energy_data, None,
									 memory_mode=tin.memory_mode, device=device, dataname="train")
    grouped_valid_data = GroupedDataset(valid_energy_data, None,
									    memory_mode=tin.memory_mode, device=device, dataname="valid")
    
    grouped_train_loader = DataLoader(grouped_train_data, batch_size=1, shuffle=False,
                                  collate_fn=custom_collate, num_workers=0)
    grouped_valid_loader = DataLoader(grouped_valid_data, batch_size=1, shuffle=False,
                                    collate_fn=custom_collate, num_workers=0)

    return grouped_train_loader, grouped_valid_loader

In [11]:
diff = torch.rand((3,3)) - torch.ones((3,1))
torch.sum(diff, dim=0)

tensor([-1.4618, -1.8635, -2.4441])

In [19]:
np.random.seed(42)

dataset_size = len(list_structures_energy)
test_split = 0.2
valid_split = 0.2

indices = list(range(dataset_size))
np.random.shuffle(indices)
split   = int(np.floor(test_split * dataset_size))

training_indices, test_indices = indices[split:], indices[:split]
training_indices, valid_indices = training_indices[split:], training_indices[:split]

In [26]:
for x in valid_indices:
    if x in training_indices:
        print('ciao')

In [16]:
from bnn import BayesianNeuralNetwork
from pyro.infer import SVI, Trace_ELBO, TraceMeanField_ELBO
from pyro.infer.autoguide import AutoDiagonalNormal
from pyro.infer import Predictive
import pyro

net = NetAtom(tin.networks_param["input_size"], tin.networks_param["hidden_size"],
			    tin.sys_species, tin.networks_param["activations"], tin.alpha, device)

bnn = BayesianNeuralNetwork(net)
guide = AutoDiagonalNormal(bnn.model)

In [17]:
pyro.clear_param_store()
num_steps = 300
initial_lr = 0.001
gamma = 0.1  # final learning rate will be gamma * initial_lr
lrd = gamma ** (1 / num_steps)
optim = pyro.optim.ClippedAdam({'lr': initial_lr, 'lrd': lrd})
svi = SVI(bnn.model, guide, optim, loss=TraceMeanField_ELBO())

for j in range(num_steps):
    loss = 0
    for data_batch in grouped_train_loader:
        grp_descrp, grp_energy, logic_reduce = data_batch[0][10], data_batch[0][11], data_batch[0][12]
        grp_descrp[0] = grp_descrp[0].float()
        grp_descrp[1] = grp_descrp[1].float()

        logic_reduce[0] = logic_reduce[0].float()
        logic_reduce[1] = logic_reduce[1].float()
        loss += svi.step(grp_descrp, logic_reduce, grp_energy) 
    if j % 100 == 0:
        print("[EPOCH LOSS %04d] loss: %.4f" % (j + 1, loss / len(logic_reduce[0])))

[EPOCH LOSS 0001] loss: 247.7633
[EPOCH LOSS 0101] loss: 3.7544
[EPOCH LOSS 0201] loss: 3.9208


In [18]:
pyro.clear_param_store()
adam = pyro.optim.Adam({"lr": 0.05})
svi = SVI(bnn.model_single, guide, adam, loss=Trace_ELBO())

X = list_structures_energy[:2]
y = [x.energy for x in X]

svi.step(X,y)

RuntimeError: expected scalar type Float but found Double
                        Trace Shapes:        
                         Param Sites:        
                        Sample Sites:        
functions.0.Linear_Sp1_F1.weight dist | 20 60
                                value | 20 60
  functions.0.Linear_Sp1_F1.bias dist | 20   
                                value | 20   

In [ ]:
import torch

a = torch.tensor([[0],[0]])
b = torch.tensor([[1,2,1],[1,2]])
c = torch.cat((a,b))
c

ValueError: expected sequence of length 3 at dim 1 (got 2)

In [ ]:
torch.sum(b, dim=0)

tensor([2, 4])